In [ ]:
# Import libraries
## Basic libs
import pandas as pd
import numpy as np
import warnings
## Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Configure libraries
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')

In [ ]:
data=pd.read_csv(r"C:\Users\mayan\OneDrive\Desktop\Placement Projects\classification project\bank.csv")

In [ ]:
data.info()

In [ ]:
# Drop 'duration' column
#data = data.drop('duration', axis=1)

# print(df_bank.info())
#print('Shape of dataframe:', data.shape)
#data.head()

In [ ]:
data.describe()

In [ ]:
def bar_plot(variable):
    var =data[variable]
    varValue = var.value_counts()
    plt.figure(figsize=(15,3))
    plt.bar(varValue.index, varValue,color=['#00008b','#00e5ee','#cd1076', '#008080','#cd5555','red','blue',])
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
categoryc = ["job","marital","education", "housing", "loan","contact","poutcome","month","deposit"]
for c in categoryc:
    bar_plot(c)

In [ ]:
def plot_hist(variable):
    plt.figure(figsize=(9,6))
    plt.hist(data[variable], bins=40,color='#cd1076')
    plt.xlabel(variable)
    plt.ylabel("frequency")
    plt.title("{} distrubition with hist".format(variable))
    plt.show()

In [ ]:
numericVar = ["age","campaign"]
for n in numericVar:
    plot_hist(n)

In [ ]:
pd.crosstab(data.age,data.deposit).plot(kind="area",figsize=(15,7),color=['#0000ff','#000000' ])
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
pd.crosstab(data.job,data.deposit).plot(kind="barh",figsize=(15,7),color=['#0000ff','#000000'])
plt.title('Deposit Age Distribution')
plt.xlabel('Frequency')
plt.ylabel('Job')
plt.show()


In [ ]:
from collections import Counter
def detect_outliers(data,features):
    outlier_indices = []
    for c in features:
        # 1st quartile
        Q1 = np.percentile(data[c],25)
        # 3rd quartile
        Q3 = np.percentile(data[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = data[(data[c] < Q1 - outlier_step) | (data[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers


In [ ]:
data.loc[detect_outliers(data,['age','day','campaign','previous'])]

In [ ]:
data.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns for correlation
numeric_data = data.select_dtypes(include=[float, int])

# Generate the heatmap
fig, ax = plt.subplots(figsize=(13, 13))
sns.heatmap(numeric_data.corr(), annot=True, linewidths=.5, ax=ax)

plt.show()


In [ ]:
def pdayswork(pdays):
    if(pdays == -1):
        return(0)
    elif(pdays >= 0):
        return(1)
data['pdays'] = data['pdays'].apply(pdayswork)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler

# Copying original dataframe
data1 = data.copy()

scaler = StandardScaler()
num_cols = ['age', 'balance', 'day', 'campaign', 'previous']
data1[num_cols] = scaler.fit_transform(data[num_cols])

data1.head()
scaler = StandardScaler()
num_cols = ['age', 'balance', 'day', 'campaign', 'previous']
data1[num_cols] = scaler.fit_transform(data[num_cols])

data1.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False)  # Updated from sparse=False to sparse_output=False for compatibility
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Encode Categorical Data
df_encoded = pd.DataFrame(encoder.fit_transform(data1[cat_cols]))
df_encoded.columns = encoder.get_feature_names_out(cat_cols)

# Replace Categorical Data with Encoded Data
data1 = data1.drop(cat_cols, axis=1)
data1 = pd.concat([df_encoded, data1], axis=1)

# Encode target value
data1['deposit'] = data1['deposit'].apply(lambda x: 1 if x == 'yes' else 0)

print('Shape of dataframe:', data1.shape)
data1.head()


In [ ]:
data1.info()

In [ ]:
data1.iloc[:,50]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, ConfusionMatrixDisplay, f1_score

# Split data
X = data1.iloc[:, 0:51]
Y = data1.iloc[:, 51]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=100)

# Initialize dictionaries
accuracies = {}
kappaScores = {}
f1scores = {}
recallScores = {}

In [ ]:
y_train.value_counts()

In [ ]:
X_train

In [ ]:
##Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, f1_score, cohen_kappa_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import numpy as np

# Optimize the pipeline for speed
pipeline = Pipeline([
    ('poly', PolynomialFeatures(interaction_only=True, include_bias=False)),  # Only interaction features
    ('logreg', LogisticRegression(max_iter=200, solver='saga', warm_start=True, n_jobs=-1))  # Fastest solver with warm start
])

# Define the parameter distribution
param_distributions = {
    'poly__degree': [3],  # Reduce range of polynomial degrees to speed up
    'logreg__C': np.logspace(-2, 2, 6),  # Slightly reduce the grid size
    'logreg__penalty': ['l1', 'l2', 'elasticnet'],  # Focus on penalties supported by saga solver
    'logreg__max_iter': [300]  # Limit max_iter range
}

# RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=param_distributions, 
                                   n_iter=30,  # Reduce the number of random combinations
                                   cv=3,  # Reduce the number of folds for faster execution
                                   n_jobs=-1, 
                                   scoring='accuracy',
                                   random_state=42,
                                   verbose=3)  # Set verbosity to 3 to print progress for each iteration

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters found: ", random_search.best_params_)

# Predict using the best model
best_model = random_search.best_estimator_
predictions_test = best_model.predict(X_test)
predictions_train = best_model.predict(X_train)

# Test accuracy
test_acc = accuracy_score(y_test, predictions_test) * 100
print("Test Accuracy:", test_acc)

# Train accuracy
train_acc = accuracy_score(y_train, predictions_train) * 100
print("Train Accuracy:", train_acc)

# Recall score on test data
recall = recall_score(y_test, predictions_test, average='weighted') * 100
print("Recall Score:", recall)

# F1-Score on test data
f1 = f1_score(y_test, predictions_test, average='weighted') * 100
print("F1-Score:", f1)

# Cohen Kappa score on test data
cohen_kappa = cohen_kappa_score(y_test, predictions_test) * 100
print('Cohen Kappa Score:', cohen_kappa)

# Classification report on test data
print("Classification Report:")
print(classification_report(y_test, predictions_test))

# Confusion matrix
cm = confusion_matrix(y_test, predictions_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(cmap='Blues')



In [ ]:
#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap=True, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 2000)

clf.fit(X_train,y_train)

prediction = clf.predict(X_test)
acc = accuracy_score(y_test,prediction)*100
print("Random Forest accuracy:",acc)
accuracies['Random Forest']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Random Forest']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Random Forest']=cohen_kappa

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, cohen_kappa_score
import numpy as np

# Define a more focused parameter grid
param_grid = {
    'n_estimators': [100, 2000],  # Reduced range
    'max_depth': [10, 20, 30],   # Reduced range
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Initialize the RandomizedSearchCV with reduced iterations and verbosity
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid,
                                   n_iter=20, cv=5, verbose=2, random_state=50, n_jobs=-1)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters from RandomizedSearchCV
print("\nBest parameters found: ", random_search.best_params_)

# Train accuracy
train_pred = random_search.predict(X_train)
train_acc = accuracy_score(y_train, train_pred) * 100
train_recall = recall_score(y_train, train_pred) * 100
print("\nTrain Accuracy: {:.2f}%".format(train_acc))
print("Train Recall: {:.2f}%".format(train_recall))

# Test accuracy
test_pred = random_search.predict(X_test)
test_acc = accuracy_score(y_test, test_pred) * 100
test_recall = recall_score(y_test, test_pred) * 100
print("\nTest Accuracy: {:.2f}%".format(test_acc))
print("Test Recall: {:.2f}%".format(test_recall))

# Store accuracies and recall scores
accuracies['Random Forest'] = test_acc
recall_scores['Random Forest'] = test_recall

# F1-Score
f1 = f1_score(y_test, test_pred) * 100
print("\nF1-Score: {:.2f}%".format(f1))
f1scores['Random Forest'] = f1

# Cohen Kappa Score
cohen_kappa = cohen_kappa_score(y_test, test_pred) * 100
print("Cohen Kappa score: {:.2f}%".format(cohen_kappa))
kappaScores['Random Forest'] = cohen_kappa


In [ ]:
#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(X_train,y_train)
naiveb=nb.predict(X_test)
prediction= nb.predict(X_test)
acc = accuracy_score(y_test,prediction)*100
print("Naive Bayes accuracy:",acc)
accuracies['Naive Bayes']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Naive Bayes']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Naive Bayes']=cohen_kappa

In [ ]:
###NB

In [ ]:
# Training Naive bayes
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report

gnb = GaussianNB()
gnb.fit(X_train, y_train)
pred = gnb.predict(X_test)
accNB = accuracy_score(y_test, pred)
y_pred_prob = gnb.predict_proba(X_test)
aucScoreNB = roc_auc_score(y_test,  y_pred_prob[:,1])
fprNB, tprNB, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for NB is ",aucScoreNB)
print("Test Accuracy score for NB is ",accuracy_score(y_test, pred))
predT=gnb.predict(X_train)
print("Train Accuracy score for NB is ",accuracy_score(y_train, predT))
#print("Best parameters for NB are ",gnb.best_params_)
print("Recall score for NB is ",recall_score(y_test, pred))

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier(loss='modified_huber',shuffle=True,random_state=100,penalty='l1',alpha=0.004
                  ,max_iter=100,eta0=0.2,learning_rate='optimal')
sgd.fit(X_train,y_train)
prediction=sgd.predict(X_test)
acc = accuracy_score(y_test,prediction)*100
print("SGD Classifier accuracy:",acc)
accuracies['SGDC']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['SGDC']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['SGDC']=cohen_kappa

In [ ]:
###KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors =3,algorithm='ball_tree')
knn.fit(X_train, y_train)
prediction=knn.predict(X_test)
acc = accuracy_score(y_test,prediction)*100
print("Knn accuracy:",acc)
accuracies['KNN']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['KNN']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['KNN']=cohen_kappa

In [ ]:
#########SVM

In [ ]:
classifier = SVC(probability=True)
parameters=[{'C':[1,10,100,1000],'kernel':['linear']},
            {'C':[1,10,100,1000],'kernel':['rbf'], 'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},
            {'C':[1,10,100,1000],'kernel': ['poly'], 'degree':[2,3,4]}      ]

In [ ]:
clf = GridSearchCV(estimator=classifier,param_grid=parameters,scoring='accuracy',cv=5)
best_clf = clf.fit(X_train,y_train)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Define the SVC classifier
classifier = SVC(probability=True)

# Define the parameter distributions
param_distributions = {
    'C': [1, 10, 100, 1000],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': uniform(0.1, 0.9),  # Using uniform distribution for gamma
    'degree': [2, 3, 4]
}

# Instantiate RandomizedSearchCV
clf = RandomizedSearchCV(estimator=classifier, 
                         param_distributions=param_distributions, 
                         n_iter=20,  # Number of parameter settings sampled
                         scoring='accuracy', 
                         cv=5, 
                         random_state=42, 
                         n_jobs=-1)  # Use all processors

# Fit the model
best_clf = clf.fit(X_train, y_train)

# Output the best parameters
print("Best parameters found: ", best_clf.best_params_)


In [ ]:
# Training SVM
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
# get best parameters for retraining

pred = best_clf.predict(X_test)
accSVM = accuracy_score(y_test, pred)
y_pred_prob = best_clf.predict_proba(X_test)
aucScoreSVM = roc_auc_score(y_test,  y_pred_prob[:,1])
fprSVM, tprSVM, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("AUC score for SVM is ",aucScoreSVM)
print("Test Accuracy score for SVM is ",accuracy_score(y_test, pred))
predT=best_clf.predict(X_train)
print("Train Accuracy score for SVM is ",accuracy_score(y_train, predT))
print("Best parameters for SVM are ",best_clf.best_params_)

In [ ]:
predR = best_clfR.predict(X_test)
predRT=best_clfR.predict(X_train)
recallSVM=recall_score(y_test, predR)
print("Test Recall score for SVM is ",recallSVM)
print("Train recall score for SVM is ",recall_score(y_train, predRT))
print("Best parameters for recall of SVM are ",best_clfR.best_params_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree= DecisionTreeClassifier(criterion='gini',max_depth=10,random_state=100,min_samples_leaf=10)
dtree.fit(X_train, y_train)
prediction=dtree.predict(X_test)
acc = accuracy_score(y_test,prediction)*100
print("Decision Tree accuracy:",acc)
accuracies['Decision Tree']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Decision Tree']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Decision Tree']=cohen_kappa

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.8,
    max_depth=2, random_state=0)
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
acc = accuracy_score(y_test, prediction)*100
print("Gradient Boosting Classifier accuracy:",acc)
accuracies['Gradient Boosting']=acc

f1=f1_score(y_test,prediction)*100
print("F1-Score: ",f1)
f1scores['Gradient Boosted']=f1

cohen_kappa = cohen_kappa_score(y_test, prediction)*100
print('Cohen Kappa score: ',cohen_kappa)
kappaScores['Gradient Boosting']=cohen_kappa

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, cohen_kappa_score

# Initialize the XGBoost classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.08, gamma=0, 
                    subsample=0.78, colsample_bytree=1, max_depth=7)

# Fit the model to the training data
xgb.fit(X_train, y_train)

# Predict the labels for the test set
prediction = xgb.predict(X_test)

# Calculate accuracy
acc = accuracy_score(y_test, prediction) * 100
print("XGBoost Classifier accuracy:", acc)
accuracies['XGBoost Classifier'] = acc

# Calculate F1 score
f1 = f1_score(y_test, prediction) * 100
print("F1 Score: ", f1)
f1scores['XGBoost Classifier'] = f1

# Calculate Recall score
recall = recall_score(y_test, prediction) * 100
print("Recall Score: ", recall)
recallScores['XGBoost Classifier'] = recall

# Calculate Cohen Kappa score
cohen_kappa = cohen_kappa_score(y_test, prediction) * 100
print('Cohen Kappa score: ', cohen_kappa)
kappaScores['XGBoost Classifier'] = cohen_kappa


In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("Accuracy %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(accuracies.keys()), y=list(accuracies.values()), palette=colors)
plt.show()

In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("Kappa Score %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(kappaScores.keys()), y=list(kappaScores.values()), palette=colors)
plt.show()

In [ ]:
colors = ["#00008b", "#00e5ee", "#cd1076", "#008080","#cd5555",'black']

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,3))
plt.ylabel("F1 Score %")
plt.xlabel("\n\n Algorithms")
sns.barplot(x=list(f1scores.keys()), y=list(f1scores.values()), palette=colors)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define the model
model = XGBClassifier()

# Define the parameter grid
param_grid = {
            'eta': np.arange(0.1, 0.26, 0.05),
            'min_child_weight': np.arange(1, 5, 0.5).tolist(),
            'gamma': [5],
            'subsample': np.arange(0.5, 1.0, 0.11).tolist(),
            'colsample_bytree': np.arange(0.5, 1.0, 0.11).tolist()
        }

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)


In [ ]:
# Assuming 'random_search' is the RandomizedSearchCV object used for hyperparameter tuning

# Extracting the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Training the model with the best parameters
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

# Evaluating the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

print(f"Accuracy: {accuracy:}")
print(f"F1 Score: {f1:.2f}")
print(f"Cohen's Kappa Score: {kappa:.2f}")



# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.show()


In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance  # Import plot_importance separately

# Assuming you have trained an XGBoost model
# model = xgb.XGBClassifier(...)

# Plot feature importance
plot_importance(model, importance_type='weight')
plt.show()


In [ ]:
X_train['previous']


In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Assuming you have trained a model and X_train is your training data
# model = xgb.XGBClassifier(...)
# X_train = ...

# Create PDP for specific features
features = ['previous']  # Replace with indices or names of features
fig, ax = plt.subplots(figsize=(12, 6))
PartialDependenceDisplay.from_estimator(model, X_train, features, ax=ax)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df contains your data with a binary target variable 'subscribed'
# and a predictor 'predictor_name'

# Bin the predictor values if needed
data['predictor_binned'] = pd.qcut(data['previous'], q=10)  # Binning into 10 quantiles

# Calculate conversion rate by predictor value
conversion_rate = data.groupby('previous')['deposit'].mean()

# Plot conversion rates
conversion_rate.plot(kind='bar')
plt.xlabel('Predictor Value Binned')
plt.ylabel('Conversion Rate')
plt.title('Conversion Rate by Predictor Value')
plt.show()


In [ ]:
from sklearn.inspection import partial_dependence

# Example: Partial dependence for 'balance'
pd_results = partial_dependence(
    model, X_train, features=[X_train.columns.get_loc('balance')]
)

# Extract the partial dependence values
pd_values = pd_results['average']  # For scikit-learn version 0.24 and later

print("Partial Dependence values for 'balance':")
print(pd_values)


In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Assuming 'model' is your trained model and 'X_train' is your training data

features = ['previous', 'poutcome_success']  # Replace with the features you want to analyze
PartialDependenceDisplay.from_estimator(model, X_train, features, grid_resolution=20)

plt.suptitle('Partial Dependence of Balance and Age on Subscription Probability')
plt.show()


In [ ]:
import pandas as pd

# Bin features if necessary
data1['balance_binned'] = pd.qcut(data['balance'], q=10)
data1['age_binned'] = pd.qcut(data['age'], q=10)

# Calculate conversion rates
combination_rates = data1.groupby(['balance_binned', 'age_binned'])['deposit'].mean().reset_index()

# Pivot table for heatmap
pivot_table = combination_rates.pivot('balance_binned', 'age_binned', 'deposit')

# Plot
plt.figure(figsize=(10, 8))
plt.title('Conversion Rate by Combination of Balance and Age')
plt.xlabel('Age Binned')
plt.ylabel('Balance Binned')
plt.imshow(pivot_table, aspect='auto', cmap='viridis')
plt.colorbar(label='Conversion Rate')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample DataFrame (replace with your actual data)
# df = pd.read_csv('your_data.csv')

# Binning the balance into quantiles
data1['balance_binned'] = pd.qcut(data1['balance'], q=10, labels=False)

# Calculate conversion rate by balance bin and credit default status
conversion_rate = data1.groupby(['balance_binned', 'default_yes'])['deposit'].mean().unstack()

# Plotting
conversion_rate.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Balance Bin')
plt.ylabel('Conversion Rate')
plt.title('Conversion Rate by Balance and Credit Default Status')
plt.legend(title='Credit Default')
plt.show()


In [ ]:
import pandas as pd

# Sample DataFrame (replace with your actual data)
# df = pd.read_csv('your_data.csv')

# Bin the balance into quantiles
bins = pd.qcut(data1['balance'], q=10, retbins=True)[1]  # Get bin edges

# Print the bin edges
print("Bin edges for 'balance':")
print(bins)


In [ ]:
# Assuming bins have been calculated as above
bin_index = 6  # Bin 7 corresponds to index 6

# Define the range for bin 7
bin_range = (bins[bin_index], bins[bin_index + 1])

# Filter the data that falls into bin 7
df_bin7 = data1[(data1['balance'] > bin_range[0]) & (data1['balance'] <= bin_range[1])]

# Display the data for bin 7
print(f"Data for bin 7 (Balance between {bin_range[0]} and {bin_range[1]}):")
print(df_bin7)

In [ ]:
x=data['balance'].mean()

In [ ]:
y=data['balance'].var()

In [ ]:
z=y**(1/2)

In [ ]:
-0.2064753533504524*z+x


In [ ]:
-0.058239648485650324*z+x

In [ ]:
import shap

# Calculate SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test)

# Plot SHAP dependence plot for a specific feature
shap.dependence_plot("feature_name", shap_values, X_test)


In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp

# Define the hyperparameter space
space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1)
}

# Define the objective function to minimize
def objective(params):
    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

# Perform the optimization
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

In [ ]:
pip install hypercot